# Исследование надежности заемщиков

Цель работы - разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Изучение общей информации о данных 

In [1]:
import pandas as pd# импортируем библиотеку pandas

In [2]:
data=pd.read_csv('проект 2.csv')#открытие файла
data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [3]:
data.info()# вывод информации файле

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Данные показывают, что у нас есть пропуски в столбце days_employed и total_income, которые мы обработаем в следующем блоке. Там же мы изменим тип данных, где это нужно.


## Предобработка данных

### Удаление пропусков

In [4]:
#вывод количества пропуском
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

total_income — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполним пропуски в этом столбце медианным значением по каждому типу из столбца income_type.

In [5]:
for ind in data['income_type'].unique():
    data.loc[(data['income_type']==ind)&(data['total_income'].isna()),'total_income']=\
    data.loc[(data['income_type']==ind),'total_income'].median()
print(data['total_income'].isna().sum())

0


### Обработка аномальных значений

In [6]:
#замена всех отрицательных значений на положительные
data['days_employed']=abs(data['days_employed'])

In [7]:
data.groupby('income_type')['days_employed'].median()# вывод медианного значения трудового стажа days_employed в днях.

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть.

In [8]:
#вывод уникальных значений в столбце 
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

In [9]:
#оставляем данные без аномалий
data=data.loc[(data['children']>=0)&(data['children']<20)]

In [10]:
#вывод уникальных значений
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

### Удаление пропусков (продолжение)

Заполним пропуски в столбце days_employed медианными значениями по каждому типу занятости income_type

In [11]:
#заполнение пропусков медианным значением 
for ind in data['income_type'].unique():
    data.loc[(data['income_type']==ind)&(data['days_employed'].isna()),'days_employed']=\
    data.loc[(data['income_type']==ind),'days_employed'].median()
print(data['days_employed'].isna().sum())

0


In [12]:
#проверка данных на пропуски
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21402 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21402 non-null  int64  
 1   days_employed     21402 non-null  float64
 2   dob_years         21402 non-null  int64  
 3   education         21402 non-null  object 
 4   education_id      21402 non-null  int64  
 5   family_status     21402 non-null  object 
 6   family_status_id  21402 non-null  int64  
 7   gender            21402 non-null  object 
 8   income_type       21402 non-null  object 
 9   debt              21402 non-null  int64  
 10  total_income      21402 non-null  float64
 11  purpose           21402 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


### Изменение типов данных

In [13]:
data['total_income']=data['total_income'].astype('int')#изменение типа данных

### Обработка дубликатов

Обработаем неявные дубликаты в столбце education и приведем все названия к нижнему регистру

In [14]:
data['education']=data['education'].str.lower()#приведение данных в столбце к нижнему регистру

In [15]:
data.duplicated().sum() #подсчет дубликатов

71

In [16]:
data=data.drop_duplicates()# удаляем дубликаты

### Категоризация данных

In [17]:
def categorize_income(row):
    total_income=row['total_income']
    if total_income<=30000:
        return 'E'
    if total_income <=50000:
        return 'D'
    if total_income <=200000:
        return 'C'
    if total_income <=1000000:
        return 'B'
    return 'A'# создание функции categorize_income() для отнесения людей по их доходу на группы

In [18]:
import warnings 
warnings.filterwarnings("ignore") #убираем красные текстовые предупреждения

In [19]:
data['total_income_category']=data.apply(categorize_income,axis=1)#применяем функцию методом apply()

In [20]:
data['purpose'].unique() #перечень уникальных целей взятия кредита

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [21]:
def categorize_purpose(row):
    purpose=row['purpose']
    if 'авто' in purpose:
        return 'операции с автомобилем'
    if 'жиль' in purpose or 'недвижим' in purpose:
        return 'операции с недвижимостью'
    if 'свадьб' in purpose:
        return 'проведение свадьбы'
    return 'получение образования'#создание функции categorize_purpose() для разделения на цели взятия кредита

In [22]:
data['purpose_category']=data.apply(categorize_purpose,axis=1)#применение функции методом apply()

## Исследование данных

### Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [23]:
#создание сводной таблицы 
data_children_pivot=data.pivot_table(index=['children'],values='debt',aggfunc=('count','sum','mean')) 
data_children_pivot

,count,mean,sum
children,,,
0,14091,0.075438,1063
1,4808,0.092346,444
2,2052,0.094542,194
3,330,0.081818,27
4,41,0.097561,4
5,9,0.000000,0


Исходя из полученных данных можно сделать вывод, что больше всего должников по кредиту являются клиенты с четырьмя, двумя и одним ребенком в семье. При этом чаще всего берут кредиты клиенты бездетные. 

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [24]:
#создание сводной таблицы
data_family_status_pivot=data.pivot_table(index=['family_status'],values='debt',aggfunc=('count','sum','mean'))
data_family_status_pivot

,count,mean,sum
family_status,,,
Не женат / не замужем,2796,0.097639,273
в разводе,1189,0.070648,84
вдовец / вдова,951,0.066246,63
гражданский брак,4134,0.093130,385
женат / замужем,12261,0.075606,927


Исходя из полученных данных больше всего задолжностей по кредиту в средне арифметическом значение у не женатых/не замужних и у клиентов, состоящих в гражданском браке. Из этого можно сделать вывод, что чаще всего кредиты берут женатые и незамужние клиенты, но при этом больше всего задолжностей у не женатых/не замужних и у клиентов, состоящих в гражданском браке.

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [25]:
#создание сводной таблицы 
data_total_income_pivot=data.pivot_table(index=['total_income_category'],values='debt',aggfunc=('count','sum','mean'))
data_total_income_pivot

,count,mean,sum
total_income_category,,,
A,25,0.080000,2
B,5014,0.070602,354
C,15921,0.084982,1353
D,349,0.060172,21
E,22,0.090909,2


 Исходя из полученных данных, больше всего берут кредиты клиенты с заработком от 50001-200000 рублей, при этом 8.49% клиентов имеют задолжность по кредиту. Меньше всего кредитов берут клиенты с заработком до 30000 рублей, но у них чаще всего задолжность по кредиту, которая составляет 9.09%.

### Как разные цели кредита влияют на его возврат в срок?

In [26]:
#создание сводной таблицы 
data_purpose_pivot=data.pivot_table(index=['purpose_category'],values='debt',aggfunc=('count','sum','mean'))
data_purpose_pivot

,count,mean,sum
purpose_category,,,
операции с автомобилем,4279,0.093480,400
операции с недвижимостью,10751,0.072551,780
получение образования,3988,0.092528,369
проведение свадьбы,2313,0.079118,183


Исходя из данных, чаще всего берут кредиты на операции с недвижимостью, но при этом только 7.25% имеют задолжность по кредиту. Больше всего процент задолжности по кредиту приходит на операции с автомобилем и на получение образования.

### Bозможные причины появления пропусков в исходных данных.

В исходных данных пропуски совпадают у одного и того же клиента в days_employed и total_income. У этих клиентов тип занятости пенсионер или госслужащий. Им могли выдать кредиты без подтверждения стажа и ежемесячного дохода только на небольшую сумму. Выдача кредитов без справок и подтверждения дохода становится для банка дополнительным конкурентным предложением: он может привлечь клиентов, не желающих тратить несколько дней на сбор документов. Но у этого решения есть конечно свои риски

## Вывод

Была проведена аналитическая работа по изучению влияния разных факторов на погашение кредита в срок. Входные данные от банка- статистика о платежеспособности клиентов.

Выдвинутые гипотезы для проверки данных:

1)Есть ли зависимость между количеством детей и возврата кредита в срок?

2)Есть ли зависимость между семейным положением и возвратом кредита в срок?

3)Есть ли зависимость между уровнем дохода и возврата кредита в срок?

4)Как разные цели кредита влияют на его возврат в срок?

**Гипотеза 1** подтверждена частично.

Клиенты не имеющие детей чаще всего берут кредиты, но из них только 7.54% имеют задолжность по кредиту. Из 7240 клиентов с разным количеством детей 669 клиентов имеют задолжность, что составляет 9.24% из общего объема. 

Для полного подтверждения этой гипотезы нужно проанализировать больше данных с выборкой клиентов от 3 детей и более. 

**Гипотеза 2** подтверждена.

Люди состоящие в узаконенных отношениях или бывшие в них являются более ответственными клиентами, выполняющие свои обязательства. Из их общего объема (14 401 клиент) только 1 074 клиента имеют задолжность по кредиту, что составляет 7.457% от общего объема. 

6 930 клиентов не состоявшие в узаконенных отношениях взяли кредит в нашем банке, но из них 658 клиентов имеют кредитную задолжность, что составляет 9.49% от общего объема. 

**Гипотеза 3** подтверждена частично.

По изученным данным видно, что проблемы с выплатой кредита в срок возникают у категории 'E'- уровень дохода до 30 000 руб., что составляет 9.09%.   
Так же проблемы с выплатой кредита у категории 'C'- уровень дохода от 50 001 руб. до 200 000 руб., из их общего объема (15 921 клиент) 8.49% имеют задолжность по кредиту.

Для полного подтверждения гипотезы нужно проанализировать больше данных клиентов из категории 'E','D','B','A'.


**Гипотеза 4** подтверждена.

Наиболее ответственными заемщиками являются клиенты, бравшие кредит на проведение свадьбы и операции с недвижимостью. Из них 7.25% клиентов , бравшие займ на операции с недвижимостью, имеют задолжность и 7.91% клиентов, бравшие займ на проведение свадьбы. 

Клиенты бравшие кредит на получение образования и операции с автомобилем имеют 9.3% задолжностей, по своей категории.  

**Рекомендации:**

1) Провести повторную проверку первой и третьей гипотезы при получении больших данных о клиентах в отдельных категориях;
2) Специалисты, строящие модели кредитного скоринга могут использовать вторую и четвертую гипотезу.